Abraham Tishelman-Charny, 3 March 2021.

The purpose of this notebook is to validate resonant HH UL gridpacks. This notebook wrangles data from events in a file and saves the output to a csv. This can also be used to study any [MadGraph5](https://launchpad.net/mg5amcnlo) gridpacks. Thanks to [plyhe docs](https://pydoc.net/pylhe/0.0.2/pylhe/) and [scikit-hep z peak tutorial](https://github.com/scikit-hep/pylhe/blob/master/examples/zpeak.ipynb).

Updated by Vivan Nguyen, 26 March 2021.

# Imports

In [1]:
import os

import numpy as np
import pandas as pd

import pylhe
import ROOT
from ROOT import TLorentzVector

Welcome to JupyROOT 6.12/06


# Make pyLHE file opener to close file properly

TODO: Write a close function to close files correctly.

In [2]:
class pyLHEFileOpener(file):
    def __init__(self, file_name):
        self.f = pylhe.readLHE(file_name)
    def __enter__(self):
        return self.f
    def __exit__(self, exc_type, exc_value, traceback):
        self.f.close()

# Retrieving Files

Get the LHE files assuming directory tree stree: LHE_Files/<gridpack_Type>/<files>

In [3]:
gridpack_type = 'GF_NMSSM'
#gridpack_type = "GF_Spin-0"
# gridpack_type = "GF_Spin-2"
# gridpack_type = "VBF_Spin-0"
# gridpack_type = "VBF_Spin-2"

def get_files(gridpack_type: str):
    """Returns files based on gridpack.
    
    Parameters
    ----------
    gridpack_type : str
        Type of gridpack specifying where particles come from.
        
    Returns
    -------
    files : list
        List of files to wrangle.
    """
    
    files = ['LHE_Files/%s/%s' % (gridpack_type, file)
             for file in os.listdir('./LHE_Files/%s' % (gridpack_type))
             if file.endswith('.lhe')]
    return files

files = get_files(gridpack_type)
for f in files:
    print('Found file:', f)
print('N files:', len(files))

Found file: LHE_Files/GF_NMSSM/NMSSM_XYH_bbZZ_MX_300_MY_170_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.lhe
N files: 1


# Wrangle the data

## Global parameters

The global parameters set whether or not to debug and how many events to use. Additionally, they set which variables to save for each individual particle of interest and which particle families have leading and subleading particles.

In [4]:
DEBUG = False     
MAX_EVENTS = 100

# Define variables to save for each particle
VARIABLES = ['px', 'py', 'pz', 
             'e', 'm', 'spin',
             'status', 'lifetime', 'pt']

# Define particles to determine leading and subleading
DOUBLE_PARTICLE_DICT = {'GF_NMSSM': ['H', 'g']}  

PARTICLE_DICTIONARY = {'GF_NMSSM': ['X', 'Lead_H',
                                  'Sublead_H','Lead_g',
                                  'Sublead_g']}

## Functions for event loop

In [38]:
def get_pt(p):
    """Computes the transverse momentum.
    
    Parameters
    ----------
    p : Particle
        An instance of the Particle class.
    
    Returns
    -------
    pt : np.float64
        Transverse momentum.
    """
    
    px = p.px
    py = p.py 
    pt = np.sqrt(px**2 + py**2)
    return pt

def get_delta_r(eta1: float, eta2: float, phi1: float, phi2: float):
    """Returns the delta R between two particles.
    
    Parameters
    ----------
    eta1 : float
        Eta of the first particle.
    eta2 : float
        Eta of the second particle.
    phi1 : float
        Phi of the first particle.
    phi2 : float
        Phi of the second particle.
    """
    
    dr = np.sqrt((eta1 - eta2)**2 + (phi1 - phi2)**2)
    return dr

def debug_message(particle):
    """Prints the attributes of a particle.
    
    Parameters
    ----------
    particle : Particle
        An instance of the particle class.
    
    Returns
    -------
    None
    """
    
    print('\npdgId:', particle.id)
    print('mass:', particle.m)
    for var in ['mother1', 'mother2', 'color1', 'color2',
                'status', 'lifetime', 'px', 'py', 'pz', 'spin']:
        print('%s: %s' % (var, vars(particle)[var]))
    
def wrangle_double_particles(d: dict, particle_type: str, particles):
    """Saves wrangling performed on the leading and subleading particles.
    
    Parameters
    ----------
    in_dict : dict
        Dictionary containing wrangled information about events.
    particle_type : str
        String defining whether it's a Higgs, gluon, etc. pair.
    particle : list
        List of particles
        
    Returns
    -------
    None
    """
    
    # Sort particles by descending pt.
    pts = [get_pt(particle) for particle in particles]
    if particle_type == 'H':
        sorted_idxs = np.argsort(pts)[::-1]
    else:
        sorted_idxs = np.arange(len(particles))
    lead_particle = particles[sorted_idxs[0]]
    sublead_particle = particles[sorted_idxs[1]]

    
    #  Record leading and subleading attributes.
    for idx, rank in enumerate(['Lead', 'Sublead']):
        for var in VARIABLES:
            if var == 'pt':
                d['_'.join([rank, particle_type, var])].append(pts[sorted_idxs[idx]])
            else:
                d['_'.join([rank, particle_type, var])].append(vars(particles[sorted_idxs[idx]])[var])
    
    #  Calculate delta R
    lead_tlv = TLorentzVector()
    sublead_tlv = TLorentzVector()
    
    for idx, tlv in enumerate([lead_tlv, sublead_tlv]):
        tlv.SetPxPyPzE(vars(particles[sorted_idxs[idx]])['px'],
                       vars(particles[sorted_idxs[idx]])['py'],
                       vars(particles[sorted_idxs[idx]])['pz'],
                       vars(particles[sorted_idxs[idx]])['e'])
        
    for tlv, rank in zip([lead_tlv, sublead_tlv], ['Lead', 'Sublead']):
        d['_'.join([rank, particle_type, 'eta'])].append(tlv.Eta())
        d['_'.join([rank, particle_type, 'phi'])].append(tlv.Phi())
        
    delta_eta = float(lead_tlv.Eta() - sublead_tlv.Eta())
    delta_phi = float(lead_tlv.Phi() - sublead_tlv.Phi())
    
    delta_r = get_delta_r(lead_tlv.Eta(), sublead_tlv.Eta(),
                          lead_tlv.Phi(), sublead_tlv.Phi())
    
    if particle_type == 'H':
        d['DR_H1H2'].append(delta_r)
        d['H1H2_Deta'].append(delta_eta)
        d['H1H2_Dphi'].append(delta_phi)
    elif particle_type == 'Outgoing_q':
        d['DR_OutqOutq'].append(delta_r)
        d['OutqOutq_Deta'].append(delta_eta)
        d['OutqOutq_Dphi'].append(delta_phi)
    elif particle_type == 'Incoming_q':
        d['DR_InqInq'].append(delta_r)
        d['InqInq_Deta'].append(delta_eta)
        d['InqInq_Dphi'].append(delta_phi) 
            
def wrangle_resonant_particle(d: dict, particle):
    """Saves wrangling performed on the resonant particle.
    
    Parameters
    ----------
    in_dict : dict
        Dictionary containing wrangled information about events.
    particle : Particle
        The resonant particle.
        
    Returns
    -------
    None
    """
    
    for var in VARIABLES:
        str_var = 'X_%s' % (var)
        if(var == "pt"):
            x_pt = get_pt(particle)
            d[str_var].append(x_pt)
        else:
            d[str_var].append(vars(particle)[var])

    x_tlv = TLorentzVector() 
    x_tlv.SetPxPyPzE(vars(particle)['px'], vars(particle)['py'],
                     vars(particle)['pz'], vars(particle)['e'])
    d['X_eta'].append(x_tlv.Eta())
    d['X_phi'].append(x_tlv.Phi())

## Process each file

In [39]:
def initialize_dictionary(gridpack_type: str):
    """Initialize dictionary that contains wrangle information.
    
    Parameters
    ----------
    gridpack_type : str
        Type of gridpack which specifies what is the resonant particle
        and which families of particles contain a leading and subleading pair.
        
    Returns
    -------
    d : dict
        Dictionary for saving information.
    """
    
    d = {}
    
    particle_names = PARTICLE_DICTIONARY[gridpack_type]
    double_particles = DOUBLE_PARTICLE_DICT[gridpack_type]
    
    # Initialize dictionary lists 
    for p in particle_names:
        for v in VARIABLES:
            column_name = '_'.join([p,v])
            d[column_name] = []
    
    # Save eta and phi for X, both Higgs 
    for ang_particle in ['X', 'Lead_H', 'Sublead_H', 'Lead_g', 'Sublead_g']:
        for ang_var in ['eta', 'phi','pt']:
            d['_'.join([ang_particle, ang_var])] = []
            
    # For both GF and VBF, initialize lists to save event
    # by event pdgIds, DR(H, H), Deta(HH), Dphi(HH)
    d['pdgIds'] = []
    d['DR_H1H2'] = []
    d['H1H2_Deta'] = []
    d['H1H2_Dphi'] = []
    
    return d

In [40]:
def process_file(in_file: str, outname: str, debug: bool = False, max_events: int = 100):
    """Wrangles information from events in a file.
    
    Parameters
    ----------
    in_file : str
        Name of the file to wrangle.
    outname : str
        Name of file to save to.
    debug : bool, optional
        Specifies whether or not to print each particle's information in an event.
    max_events : int, optional
        Provides a limit to how many events in a file should be wrangled.
        
    Returns
    -------
    None
    """
    
    d = initialize_dictionary(gridpack_type)
    
    with pyLHEFileOpener(in_file) as f:
        for eidx, event in enumerate(f):
        #for eidx, event in enumerate(pylhe.readLHE(in_file)):
            # TODO: Implement module which has a progress bar like tqdm,
            # which shows completion progress and also estimated time left?
            # Then you would have the following without the next two lines:
            # for eidx, event in enumerate(tqdm(pylhe.readLHE(in_file))):
            if eidx % 1000 == 0:
                print('On event', eidx)
            if eidx == max_events:
                print('Stopping at %s events' % (max_events))
                break

            #  Place the debug check outside the other for loop,
            #  so it is checked only once. Who cares if you're
            #  looping over all the particles again. It's going to
            #  take longer anyway since we're printing stuff.
            if debug:
                for particle in event.particles:
                    debug_message(particle)

            pdg_ids = [particle.id for particle in event.particles]
            np_pdg_ids = np.array(pdg_ids, dtype=np.int16)
            # This avoids all the unnecessary looping over particles
            # which are not True for any of the if statements.
            # TODO: There's probably a better way of doing this
            #       than typing them in by hand that doesn't suffer
            #       in computational time. Using sums doesn't seem
            #       to be the way to go because they check conditions individually.
            special_idxs = np.where((np_pdg_ids == 21)
                                    | (np_pdg_ids == 25)
                                    | (np_pdg_ids == 35)
                                    | (np_pdg_ids == 45))[0]

            gluons = []
            higgs_bosons = []

            for special_idx in special_idxs:
                particle = event.particles[special_idx]

                if (particle.id == 21) and ("GF" in gridpack_type):
                    gluons.append(particle)
                elif particle.id == 25:
                    higgs_bosons.append(particle)
                elif particle.id == 35:
                    higgs_bosons.append(particle)
                # Spin-0 or Spin-2 Resonant Particle, 
                # X->YH resonant particle
                elif particle.id == 45:
                    resonant_particle = particle 

            wrangle_double_particles(d, 'H', higgs_bosons)
            wrangle_double_particles(d, 'g', gluons)                   
            wrangle_resonant_particle(d, resonant_particle)

            d['pdgIds'].append(pdg_ids)
    pd.DataFrame(d).to_csv(outname, index=False)

In [54]:
outname = 'new.csv'

for f in files:
    test=process_file(f, outname, debug=DEBUG, max_events=MAX_EVENTS)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-54-395a45db825e>", line 4, in <module>
    test=process_file(f, outname, debug=DEBUG, max_events=MAX_EVENTS)
  File "<ipython-input-40-5df7a214f263>", line 23, in process_file
    for eidx, event in enumerate(f):
  File "/usr/local/lib/python3.6/site-packages/pylhe/__init__.py", line 168, in readLHE
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/xml/etree/ElementTree.py", line 1242, in iterparse
OSError: [Errno 24] Too many open files: 'LHE_Files/GF_NMSSM/NMSSM_XYH_bbZZ_MX_300_MY_170_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.lhe'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 1863, in

OSError: [Errno 24] Too many open files: 'LHE_Files/GF_NMSSM/NMSSM_XYH_bbZZ_MX_300_MY_170_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.lhe'

# Read the output

Because the 'pdgIds' column contains lists, you will need to convert that so it is read in correctly by pandas as a list instead of a string.

In [42]:
df = pd.read_csv(outname, converters={'pdgIds': eval})

In [43]:
df1 = (pd.read_csv('/Users/vivannguyen/Downloads/df_300_170.csv',
                   converters={'pdgIds':eval})
       .drop('Unnamed: 0', axis=1))

In [44]:
df.iloc[0].pdgIds

[21.0, 21.0, 45.0, 35.0, 25.0]

In [45]:
df1.iloc[0].pdgIds

[21.0, 21.0, 45.0, 35.0, 25.0]